# Лекция 01. Введение в обучение с подкреплением

## Обучение с подкреплением

Обучение с подкреплением (reinforcement learning, RL).

### Фреймворк SLM-lab

- Официальный репозиторий: `https://github.com/kengz/SLM-Lab`
- База знаний: `https://slm-lab.gitbook.io/slm-lab`
- Установка: `https://slm-lab.gitbook.io/slm-lab/setup/installation`

### Цикл управления агентом в обучении с подкреплением

Обучение с подкреплением (англ. reinforcement learning) — это подход в машинном обучении, при котором агент учится, взаимодействуя с окружающей средой.

![](../../common/pictures/environment.png)

Система обучения с подкреплением реализует цикл управления с обратной связью, где агент и среда взаимодействуют и обмениваются сигналами, причем агент пытается максимизировать целевую функцию.

Введем следующие обозначения.

**Прецедент**: сигналы (тройка (кортеж))
$$
(s_t, a_t, r_t),
$$
что соответствует состоянию, действию и вознаграждению. Индекс $t=0\ldots T$ — момент времени (шаг).

**Эпизод**:
Временной горизонт от $t=0$ до момента завершения среды.

**Траектория**:
Последовательность прецедентов, или часть опыта, накопленного в течение эпизода:
$$
\tau=(s_0,a_0,r_0),(s_1,a_1,r_1),\ldots
$$

**Состояние**:
\begin{equation}
    s_t\in S,
\end{equation}
где $S$ — пространство состояний, это набор всех возможных в среде состояний.

**Действие**:
\begin{equation}
    a_t\in A,
\end{equation}
где $A$ — пространство действий.

**Вознаграждение**:
\begin{equation}
    r_t=R\left(s_t,a_t,s_{t+1}\right),
\end{equation}
где $R$ — функция вознаграждения.
	
Пространства $S, A$ и функция $R$ составляют кортежи $\left(s, a, r\right)$.

## Обучение с подкреплением как МППР

### Функция переходов

Рассмотрим как среда переходит из одного состояния в другое. Для этого введем **функцию переходов**. В обучении с подкреплением это компонент марковского процесса принятия решений (МППР).

Рассмотрим общую постановку задачи:
$$
s_{t+1}\sim P\left(s_{t+1}|\left(s_0,a_0\right),\left(s_1,a_1\right),\ldots,\left(s_t,a_t\right)\right).
$$
Тут утверждается, что на временном шаге $t$ следующее состояние $s_{t+1}$ берется из распределения вероятностей $P$, обусловенного всей историей. Вероятность перехода среды из состояния $s_t$ в состояние $s_{t+1}$ зависит от всех предыдущих состояний $s$ и действий $a$, которые имели место в данном эпизоде до этого момента. Такая постановка трудно реализуема на практике.

Преобразуем ее в МППР. Сделаем предположение, что переход в следующее состояние $s_{t+1}$ зависит только от предыдущих значений состояния $s_t$ и действия $a_t$, которое известно как **марковское свойство**. С учетом этого функция переходов примет вид
$$
s_{t+1}\sim P\left(s_{t+1}|s_t,a_t\right).
$$
Тут утверждается, что следующее состояние $s_{t+1}$ берется из распределения вероятностей $P\left(s_{t+1}|s_t,a_t\right)$.

### МППР

Представим задачу обучения с подкреплением в виде МППР. МППР определяется кортежем из четырех элементов $S, A, P(.), \mathcal R(.)$:

- $S$ — набор состояний;
- $A$ — набор действий;
- $P(s_{t+1}|s_t,a_t)$ — функция перехода состояний среды;
- $\mathcal R(s_t,a_t,s_{t+1})$ — функция вознаграждения среды.

**Замечание.** Функция переходов $P(s_{t+1}|s_t,a_t)$ и функция вознаграждений $\mathcal R(s_t,a_t,s_{t+1})$ недоступны для агентов. Агенты могут получать информацию об этих функциях только через состояния, действия и вознаграждения, воздействию которых подвергаются на данный момент в среде, то есть через кортежи $(s_t,a_t, r_t)$.

Чтобы формулировка задачи была полной, нужно формализовать понятие максимизируемой агентом целевой функции. Во-первых, опеределим **отдачу** $R(\tau)$, используя траекторию из эпизода $\tau=(s_0,a_0,r_0)\ldots(s_T,a_T,r_T)$:
$$
R(\tau)=r_0+\gamma r_1 +\gamma^2r_2+\ldots+\gamma^T r_T=\sum_{t=0}^{T}\gamma^t r_t.
$$
Здесь отдача определена как дисконтированная сумма вознаграждений на траектории, где $\gamma$ — коэффициент дисконтирования, $\gamma=[0,1]$.

Тогда **целевая функция** $J(\tau)$ становится просто математическим ожиданием отдачи по нескольким траекториям:
$$
J(\tau)=\mathbb{E}_{\tau\sim\pi}\left[R(\tau)\right]=\mathbb{E}_\tau\left[\sum_{t=0}^{T}\gamma^t r_t\right].
$$

Отдача $R(\tau)$ —  это сумма дисконтированнных вознаграждений $\gamma^{t}r_t$ за все временные шаги $t=0,\ldots,T$. Целевая функция $J(\tau)$ — это отдача, усредненная по нескольким эпизодам.

#### Цикл управления МППР

    Считаем, что агент и среда env уже созданы
    for episode = 0,...,MAX_EPISODE do
        state = env.reset()
        agent.reset()
        for t = 0,...,T do
            action = agent.act(state)
            state, reward = env.step(action)
            agent.update(action, state, reward)
            if env.done() then
                break
            end if
        end for
    end for

## Обучение функции в обучении с подкреплением

## Алгоритмы глубокого обучения с подкреплением

## Глубокое обучение для обучения с подкреплением

## Обучение с подкреплением и обучение с учителем

## Заключение